In [1]:
import keras
import cv2
import os
import glob
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
# load saliency and motion videos and split them into chunks
split = 5 # split the video every five seconds
path = '../'

In [31]:
def splitVideo(videos, split, tp):
    newdir = path+'datasets/content/'+tp+'splitby-'+str(split)+'-seconds'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    for video in videos:
        command = 'ffmpeg -i ../datasets/content/'+tp+'/'+video+' -reset_timestamps 1 -map 0 -segment_time '+str(split)+' -f segment '+newdir+'/'+video[:-4]+'%03d.mp4'
        os.system(command)

In [138]:
def extractImages(videos, split, tp):
    newdir = path+'datasets/content/'+tp+'splitby-'+str(split)+'-seconds'
    if os.path.exists(newdir):
        os.system('rm -rf '+newdir)
    os.makedirs(newdir)
    for video in videos:
        os.system('ffmpeg -i ../datasets/content/'+tp+'/'+video+' -vf fps=30 '+newdir+'/'+video[:-4]+'%d.png')

In [143]:
saliVideos = os.listdir(path+'datasets/content/saliency/')
motionVideos = os.listdir(path+'datasets/content/motion/')

extractImages(saliVideos, split, 'saliency')
extractImages(motionVideos, split, 'motion')

In [145]:
# make both saliency and motion map videos as numpy arrays
def makeVideoNumpy(videos, split, tp):
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    newdir = path+'datasets/content/'+tp+'Images'
    for video in videos:
        count = 1
        for i in range(1, totalFrames+1, imagesToRead):
            npArray = newdir+'/'+video[:-4]+str(count)+'.npy'
            count += 1
            tempArray = []
            for j in range(imagesToRead):
                imageName = newdir+'/'+video[:-4]+str(i+j)+'.png'
                image = cv2.imread(imageName, 0)
                tempArray.append(image)
            #print(np.array(tempArray).shape)
            np.save(npArray, np.array(tempArray))
makeVideoNumpy(saliVideos, split, 'saliency')
makeVideoNumpy(motionVideos, split, 'motion')

In [133]:
# make tiles information as numpy arrays
def makeTilesNumpyBackup(videosTileData, split):
    files = glob.glob('../datasets/sensory/tile/*01*.csv')
    files = sorted(files)[5:]
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    for video, f in zip(videosTileData, files):
        count = 1
        for i in range(1, totalFrames+1, imagesToRead):
            npArray = f[:-8]+str(count)+'_split_by_'+str(split)+'_seconds.npy'
            tempArray = []
            for j in range(imagesToRead):
                tempArray.append(video[i+j-1])
            #print(np.array(tempArray).shape)
            np.save(npArray, np.array(tempArray))
            count += 1

In [7]:
# make tiles information as numpy arrays
def makeTilesNumpy(video, f, split):
    fps = 30
    totalFrames = 1800
    imagesToRead = fps*split # total images to read based on split
    count = 1
    for i in range(1, totalFrames+1, imagesToRead):
        npArray = f[:-8]+str(count)+'_split_by_'+str(split)+'_seconds.npy'
        tempArray = []
        for j in range(imagesToRead):
            tempArray.append(video[i+j-1])
        print(np.array(tempArray).shape)
        np.save(npArray, np.array(tempArray))
        count += 1

In [ ]:
# encode tile information into images in banary fashion for convenience
def encodeTileInfoToImages():
    files = glob.glob('../datasets/sensory/tile/*01*.csv')
    files = sorted(files)[5:]
    tileData = []
    for f in files:
        width = 3840
        breadth = 1920
        tileSize = 192
        frames = np.zeros((1800, width*breadth), dtype=bool)
        tilesInColumn = width / tileSize
        lines = open(f, 'r')
        lines = lines.readlines()[1:]
        tilesInFrames = []
        for line in lines:
            line = line.strip().split(',')[1:]
            line = [int(i) for i in line]
            tilesInFrames.append(line)
        for i, tiles in enumerate(tilesInFrames):
            for tileNo in tiles:
                tileRowNumber = int((tileNo - 1) / tilesInColumn)
                tileColumnNumber = (tileNo - 1) % tilesInColumn
                firstPixel = tileRowNumber * width * tileSize + tileColumnNumber * tileSize
                for rowPixel in range(0, tileSize):
                    for columnPixel in range(0, tileSize):
                        frames[i][int(firstPixel + rowPixel * width + columnPixel)] = 1
            frames[i].shape = (1920, 3840)
            print(i)
        makeTilesNumpy(frames, split, f)
encodeTileInfoToImages()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161


In [ ]:
makeTilesNumpy(tileData, split)

In [50]:
width = 512
breadth = 512
tiles = [1, 14, 19, 12]
tileSize = 64
frame = np.zeros(width * breadth)
tilesInColumn = width / tileSize

for tileNo in tiles:
    tileRowNumber = int((tileNo - 1) / tilesInColumn)
    tileColumnNumber = (tileNo - 1) % tilesInColumn
    firstPixel = tileRowNumber * width * tileSize + tileColumnNumber * tileSize
    for rowPixel in range(0, tileSize):
        for columnPixel in range(0, tileSize):
            frame[int(firstPixel + rowPixel * width + columnPixel)] = 255
frame = np.reshape(frame, (breadth, width))width*
print(frame)

[[255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

plt.imshow(frame, interpolation='nearest')
plt.savefig('sample.png')